In [1]:
from tparton.t_evolution import evolve
import matplotlib.pyplot as plt
import numpy as np
from scipy.special import gamma
    
def A(a, b, g, rho):
    return (1 + g * a / (a + b + 1)) \
        * gamma(a) * gamma(b+1) / gamma(a + b + 1) \
        + rho * gamma(a + 0.5) * gamma(b + 1) / gamma(a + b + 1.5)

def pdf_u(x, eta_u=0.918, a_u=0.512, b_u=3.96, gamma_u=11.65, rho_u=-4.60):
    return eta_u / A(a_u, b_u, gamma_u, rho_u) * np.power(x, a_u) * np.power(1-x, b_u) * (1 + gamma_u * x + rho_u * np.sqrt(x))

def pdf_d(x, eta_d=-0.339, a_d=0.780, b_d=4.96, gamma_d=7.81, rho_d=-3.48):
    return pdf_u(x, eta_d, a_d, b_d, gamma_d, rho_d)
    
def vary(n, nt):
    x = np.power(10, np.linspace(np.log10(1/3000), 0, n))
    x = np.concatenate(([0], x))

    
    u = np.stack((x, pdf_u(x))).T
    d = np.stack((x, pdf_d(x))).T
    u_evolved = evolve(u, Q0_2=4, Q2=200, l_QCD=0.231, n_f=4, n_t=nt, morp='minus', logScale=True)
    d_evolved = evolve(d, Q0_2=4, Q2=200, l_QCD=0.231, n_f=4, n_t=nt, morp='minus', logScale=True)
    np.savez(f'hirai-exact_alpha_nx_{n}_nt_{nt}.npz', x=x, upd=u[:,1]+d[:,1], u_evolved=u_evolved, d_evolved=d_evolved)

for nx in [100,200,500,1000,2000,3000]:
    for nt in [25, 50, 100, 200, 500]:
        vary(nx, nt)